In [1]:
##vs branch testing

#what to do for visualizzation piece in notebooks

##vidsuaslizations fine in JN

##databricks

#nlp visualizations 
## we can do a heatmap of words or phrases that are "spammy"
##we can do a word cloud of words or phrases that are "spammy"

#unsupervised learning

In [1]:
#lets get crackn
import os
import chardet #for detecting encoding
from email import policy
from email.parser import BytesParser
#^^these two above work for MIME format^^
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import string
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer #vectoring is after preporcessing
from sklearn.model_selection import train_test_split
#download stuff for nltk if need be
#nltk.download('stopwords')
#^^this is to update stopwords^^

In [20]:
#lets build an oven (corpus)
##ham_folder_easy_uno = 'Resources/easy_ham_uno'
##spam_folder_uno = 'Resources/spam_uno'
##ham_test_folder = 'Resources/ham_testing_renaming'
#^^paths to our email datasets^^
#we need to mark which emails are spam or ham so have lists for emails and labels
emails = []
labels = []
#read the text files from our folders and give them labels
def read_files(folder, label):
    files = os.listdir(folder)
    for file in files:
        file_path = os.path.join(folder,file)
        #we ask chardet to inspect our meat
        with open(file_path,'rb') as f:
            #UTF-8 does not work, Windows-1252 works!
            ##use chardet if we have any hiccups...
            #https://stackoverflow.com/questions/436220/how-to-determine-the-encoding-of-text
            #^^windows-1252 only reads the first line...^^
            uncooked_data = f.read()
            result = chardet.detect(uncooked_data)
            encoding = result['encoding']
        #now with correct encoding, read and append
        with open(file_path, 'r', encoding=encoding) as f:
            emails.append(f.read())
            labels.append(label)
#read ham and spam
##read_files(ham_test_folder, 'test')
read_files(ham_folder_easy_uno, 'ham')
##read_files(spam_folder_uno, 'spam')

In [12]:
#ham_test_email = 'Resources/ham_testing_renaming/testingEasyHamUno00001.txt'
#lets build an oven (corpus)
#ham_folder_easy_uno = 'Resources/easy_ham_uno'
##spam_folder_uno = 'Resources/spam_uno'
ham_test_folder = 'Resources/ham_testing_renaming'
#^^paths to our email datasets^^
#we need to mark which emails are spam or ham so have lists for emails and labels
email_subjects = []
email_bodies = []
labels = []
def parse_mime_email(file_path):
    # Read the email file in binary mode
    with open(file_path, 'rb') as file:
        mime_content = file.read()
    # Parsing the MIME content
    email_message = BytesParser(policy=policy.default).parsebytes(mime_content)
    # Extract subject
    email_subject = email_message['subject']
    # Initialize a variable to hold the email body
    email_body = ""
    # Check if the email is multipart (less headaches)
    if email_message.is_multipart():
        # Iterate through each part
        for part in email_message.walk():
            # Check if the part is text/plain or text/html
            if part.get_content_type() == "text/plain":
                # Get the text content
                email_body = part.get_payload(decode=True).decode(part.get_content_charset(), 'ignore')
                break  
            elif part.get_content_type() == "text/html":
                email_body = part.get_payload(decode=True).decode(part.get_content_charset(), 'ignore')
    else:
        email_body = email_message.get_payload(decode=True).decode(email_message.get_content_charset(), 'ignore')
    return email_subject, email_body
#show it works
##subject, email_body = parse_mime_email(ham_test_email)
##print("Subject:", subject)
##print("Email Body:", email_body)
##works! now we need to see how to read all of them at the same time... we can use another function, maybe nest
#want to also include sender, but it may give bias as to who the email is...
#from and only look for those rather than patterns of the sender...
#read the text files from our folders and give them labels
def read_files(folder, label):
    files = os.listdir(folder)
    for file in files:
        file_path = os.path.join(folder,file)
        #no encoding needed!
        #we need to specify what we want using our mime parsing function
        email_subject, email_body = parse_mime_email(file_path)
        email_subjects.append(email_subject)#using email_subject variable because parse function already did it...
        email_bodies.append(email_body) #using email_body variable because parse function already did it...
        labels.append(label)
        print(email_body)
#read ham and spam
read_files(ham_test_folder, 'test')
##read_files(ham_folder_easy_uno, 'ham')
##read_files(spam_folder_uno, 'spam')
#print(email_subjects)

The Scotsman

 Thu 22 Aug 2002 

 Meaningful sentences 
 
 Tracey Lawson 
 
 
 If you ever wanted to look like "one of the most dangerous inmates in prison
 history", as one judge described Charles Bronson, nows your chance. Bronson -
 the serial hostage taker, not the movie star - has written a health and
 fitness guide in which he shares some of the secrets behind his legendary
 muscle power. 
 
 Solitary Fitness - a title which bears testament to the fact that Bronson, 48,
 has spent 24 of his 28 prison years in solitary confinement - explains how he
 has turned himself into a lean, mean, fitness machine while living 23 hours a
 day in a space just 12 feet by eight feet, on a diet of scrubs grub and at
 virtually no cost. 
 
 The book is aimed at those who want to get fabulously fit without spending a
 fortune on gym memberships, protein supplements or designer trainers, and
 starts with a fierce attack on some of the expensive myths churned out by the
 exercise industry. 
 
 "I pi

In [14]:
#lets see if the oven cooks everything evenly
#test if our list is properly read and we see the same amount of emails as we asked it to read
ham_test_df = pd.DataFrame({'email_subject':email_subjects, 'email_body':email_bodies, 'labels':labels})
##print(ham_test_df.head())
#save as csv
import csv
ham_test_df.to_csv('Resources/Labeled_CSVs/ham_testing_labeled.csv', index=False, quoting=csv.QUOTE_ALL, encoding='utf-8-sig')
#not all data is shown in csv, but good in text edit. we can try "fixing it for excel, or try json.